# Topics in Quantitative Finance - Homework 3

Assigned: Friday, July 26, 2024.
Due: **Monday, July 29, 2024** by 1PM. 

Late homework **will not be accepted**.

$$
\newcommand{\supp}{\mathrm{supp}}
\newcommand{\E}{\mathbb{E} }
\newcommand{\Eof}[1]{\mathbb{E}\left[ #1 \right]}
\def\Cov{{ \mbox{Cov} }}
\def\Var{{ \mbox{Var} }}
\newcommand{\1}{\mathbf{1} }
\newcommand{\PP}{\mathbb{P} }
\newcommand{\Pof}[1]{\mathbb{P}\left[ #1 \right]}
%\newcommand{\Pr}{\mathrm{Pr} }
\newcommand{\QQ}{\mathbb{Q} }
\newcommand{\RR}{\mathbb{R} }
\newcommand{\DD}{\mathbb{D} }
\newcommand{\HH}{\mathbb{H} }
\newcommand{\spn}{\mathrm{span} }
\newcommand{\cov}{\mathrm{cov} }
\newcommand{\sgn}{\mathrm{sgn} }
\newcommand{\HS}{\mathcal{L}_{\mathrm{HS}} }
%\newcommand{\HS}{\mathrm{HS} }
\newcommand{\trace}{\mathrm{trace} }
\newcommand{\LL}{\mathcal{L} }
%\newcommand{\LL}{\mathrm{L} }
\newcommand{\s}{\mathcal{S} }
\newcommand{\ee}{\mathcal{E} }
\newcommand{\ff}{\mathcal{F} }
\newcommand{\hh}{\mathcal{H} }
\newcommand{\bb}{\mathcal{B} }
\newcommand{\dd}{\mathcal{D} }
\newcommand{\g}{\mathcal{G} }
\newcommand{\p}{\partial}
\newcommand{\half}{\frac{1}{2} }
\newcommand{\T}{\mathcal{T} }
\newcommand{\bi}{\begin{itemize}}
\newcommand{\ei}{\end{itemize}}
\newcommand{\beq}{\begin{equation}}
\newcommand{\eeq}{\end{equation}}
\newcommand{\beas}{\begin{eqnarray*}}
\newcommand{\eeas}{\end{eqnarray*}}
\newcommand{\cO}{\mathcal{O}}
\newcommand{\cF}{\mathcal{F}}
\newcommand{\cL}{\mathcal{L}}
\newcommand{\BS}{\text{BS}}
$$

<font color = "red">Homework is to be done by each student individually.  To receive full credit, you must email a completed copy of this Jupyter notebook to TA 郭鵬 at [g.p@pku.edu.cn](mailto:g.p@pku.edu.cn) by the due date and time.  All codes must run correctly and solutions must be written up neatly in Markdown/LaTeX format. If you encounter problems with Jupyter notebook, please contact TA [郭鵬](mailto:g.p@pku.edu.cn) or [丁宏駿](dinghongjun@stu.pku.edu.cn).

## Name: <font color=blue>刘任达</font>

## Delta and delta-gamma hedging

### 1. (25 points)

A portfolio consists of 
- a long position in 500 shares of a nondividend paying stock with spot price $\$20$,
- a short position in 1000 puts struck at $\$25$ and expiring in 3 months on the stock, assumed lognormally distributed with $30\%$ volatility, 
- $\$10,000$ in money account with annual interest rate $4\%$ continuously compounding.

Answer the following questions.

* (a) What is the value of the portfolio?
* (b) How do you adjust the holdings of stock shares and cash amounts in the portfolio in order to make it delta neutral without changing the postion in puts?
* (c) How do you adjust the portfolio in order to make it delta-gamma neutral by adding position in calls struck at $\$30$? Position in puts cannot be altered. 
* (d) A month later the stock goes up to $\$24$. Determine the value of the delta-neuralized portfolio in (b). 
* (e) How do you rebalance the portfolio in (d) so it remains delta neutral?    


You may consider using the code provided in the cell below for the calculation of deltas and gammas of call and put. 

In [1]:
# as always, import required modules and functions
import numpy as np
from numpy import sqrt, log, exp
import matplotlib.pyplot as plt
import scipy.stats as ss
from scipy.stats import norm
import seaborn as sns

In [3]:
# Black-Scholes formulas
# call
def bs_call(s, K, sigma, t, r=0, d=0):
    d1 = (log(s/K) + (r - d)*t)/(sigma*sqrt(t)) + sigma*sqrt(t)/2
    d2 = d1 - sigma*sqrt(t)
    
    c = s*exp(-d*t)*norm.cdf(d1) - K*exp(-r*t)*norm.cdf(d2)
    delta = exp(-d*t)*norm.cdf(d1)
    gamma = norm.pdf(d1)/s/sigma/sqrt(t)
    
    return {'c': c, 'delta': delta, 'gamma': gamma}

#put
def bs_put(s, K, sigma, t, r=0, d=0):
    d1 = (log(s/K) + (r - d)*t)/(sigma*sqrt(t)) + sigma*sqrt(t)/2
    d2 = d1 - sigma*sqrt(t)
    
    p = K*exp(-r*t)*norm.cdf(-d2) - s*exp(-d*t)*norm.cdf(-d1)
    delta = -exp(-d*t)*norm.cdf(-d1)
    gamma = norm.pdf(d1)/s/sigma/sqrt(t)
    
    return {'p': p, 'delta': delta, 'gamma': gamma}

## <font color=blue> Solution 1. </font>
#### <font color=blue>Problem (1a)</font>

In [28]:
# (1a) 
s = 20
K, sig, t = 25, 0.3, 1/4
r = 0.04
x, y, B = 500, -1000, 20000

p, delta_p, gamma_p = bs_put(s, K, sig, t, r).values()

# The value of the portfolio is the value of all three combined

value = s*x + p*y + B
print(value)
#print(delta_p)

25132.151851511455


#### <font color=blue>Problem (1b)</font>
We know the portfolio value and delta of the portfolio
$$
\Pi = x S + y P + B \implies \Delta_{\Pi} = x + y \cdot \Delta_p
$$

To make the portfolio delta neutral, $\Delta_\Pi$ must be 0 after adjustment, so we need to sell $\Delta_\Pi$ amount of stock to even out the existing $\Delta_\Pi$:

$$
\Pi' = x S + y \cdot \Delta_P + B + (- \Delta_\Pi \cdot S  + \Delta_\Pi S) = (x - \Delta_\Pi)\cdot S + y \cdot \Delta_P + (\Delta_\Pi S + B) = x'S+y P + B'
$$

In which $\Delta_\Pi$, the new $x', B'$ respectively equal to: 

In [24]:
# (1b)
# delta per share of stock
delt = x + y * delta_p
print(delt)
x_0 = x - delt
print(x - delt)
B_0 = B + s*delt
print(B + s*delt)

1410.8418091517815
-910.8418091517815
48216.83618303563


#### <font color=blue>Problem (1c)</font>

Gamma of portfolio $\Pi$ can be written as

$$
  \Gamma_{\Pi}= x \Gamma_S + y \Gamma_P + B \Gamma_B = y \Gamma_P.
$$

In [13]:
# gamma of the portfolio
print(y*gamma_p)

-53.75326032580033


We need to add into the portfolio an instrument that has nonzero gamma, add the wanted call option $C$ to construct a new portfolio.

$$
\Pi_1 = \Pi + z\cdot C = x \cdot S + y \cdot P + + z \cdot C + (B - zC_0) \implies \Gamma_{\Pi_1} = y \Gamma_P + z \Gamma_C
$$

$\displaystyle \Gamma_{\Pi_1} = 0 \implies z = - \frac{y\Gamma_P}{\Gamma_C}$

Now we calculate $z$:

In [16]:
# (1c)
c, delta_c, gamma_c = bs_call(s, 30, sig, 1/4, r).values()
z = -y*gamma_p/gamma_c
print(z)

10747.488204943242


We then apply the delta-neutral strategy:

$$
\Delta_{\Pi_1} = x + y \Delta_P + z \Delta_C = \Delta_\Pi + z \cdot \Delta_C
$$

and construct the delta-neutral portfolio $\Pi_2$:

$$
\Delta_{\Pi_2} = (x - \Delta_{\Pi_1}) \cdot S + y \cdot P + z \cdot C + (B - zC_0 + \Delta_{\Pi_1}S_0) = x_2 S + yP + B_2
$$

Here $\Delta_{\Pi_1},\ x_2,\ B_2$ respectively equal to:

In [20]:
# (1c)

delta_pi1 = delt + delta_c * z
print(delta_pi1)
x_2 = x - delta_pi1
print(x_2)
B_2 = B - z*30 + delta_pi1*s
print(B_2)
#print(delt + z * delta_c - delta_pi1)

1466.8582707477146
-966.8582707477146
-273087.48073334293
0.0


#### <font color=blue>Problem (1d)</font>

In [31]:
# (1d)

s_new = 24

p_new, delta_p_new, gamma_p_new = bs_put(s_new, 25, sig, 1/6, r).values() #1/6 = 2/12 a year = 3-1 = 2 months
#print(p_new)

# Recalculate the delta-neutral portfolio:

newvalue = x_0 * s_new + y * p_new + B_0 * (1 + r) ** (1/12)
print(newvalue)
#print(value)

24859.2638406436


#### <font color=blue>Problem (1e)</font>
$\Delta_{\Pi_3} = x' + y \Delta_{P}$:

In [32]:
# (1e)
delta_pi3 = x_0 + y*delta_p_new
print(delta_pi3)

-324.69667925160877


$\Delta_{\Pi_3} \neq 0$, we apply delta-neutral techniques - selling $\Delta_{\Pi_3}$ shares of stock:

$\Pi_4 = (x_0 - \Delta_{\Pi_3}) S + y P + (B_0 + \Delta_{\Pi_3} S) = x_4S + yP + B_4$

where $x_4,\ B_4$ equals to: 

In [33]:
x_4 = x_0 - delta_pi3
print(x_4)
B_4 = B_0 + delta_pi3 * s_new
print(B_4)

-586.1451299001727
40424.11588099702


## The Bachelier model

### 2. (10 points)

Recall that the Bachelier model, assuming zero risk free and dividend rates, reads

$$
S_t = S_0 + \sigma W_t
$$

where $W_t$ is a Brownian motion and $\sigma$ is a constant. 

- (a) Show that a call option struck at $K$, expiring at $T$ has the price $c$ at time zero given by 

    \begin{align*}
    c(K, T) &=& \E[(S_T - K)^+]\\
    &=& \frac{\sigma\sqrt T}{\sqrt{2\pi}} \, e^{-\frac{(S_0 - K)^2}{2\sigma^2 T}} + (S_0 - K) \, N\left(\frac{S_0 - K}{\sigma\sqrt T}\right).
    \end{align*}

- (b) Verify the Dupire's formula, in this case 

    $$
    \frac{\sigma^2}{K^2} = \frac{c_T}{\frac12 K^2 c_{KK}}
    $$

    using the call price function $c$ obtained in (a). Note that subindices refer to corresponding partial derivatives. 

### <font color=blue> Solution 2(a). </font>


$$
  c\left(K, T\right) = \E\left[\left(S_t-K\right)^+\right]= \E [\left(S_0+\sigma W_T-K\right)^+ ].\textrm{ Here } W_T = \sqrt{T}Z, \text{ where } Z \sim N(0,1)
$$
Let the p.d.f. of a standard normal distribution be $\displaystyle f(x)=\frac{1}{\sqrt{2\pi}} e^{-\frac{x^2}{2}}$, the p.d.f. of $\sigma W_T$ is $g(x)$, we note that $\displaystyle g(x) = \frac{1}{\sqrt{2\pi} \sigma \sqrt{T}} e^{- \frac{x^2}{2\sigma^2 T}}$, $\displaystyle \int_{-\infty}^{x} g(s) \mathrm d s = N(\frac x {\sigma \sqrt{T}})$. 



$$
\begin{aligned}
    c\left(K, T\right) = \E\left[\left(S_0+\sigma W_T-K\right)^+\right] &= \E [ \left(S_0+\sigma W_T-K\right) 1_{\{\sigma W_T > K - S_0\}} ]\\
    & = \int_{K-S_0}^{\infty} (S_0+ x-K) g(x) \mathrm d x\\
    & = (S_0 - K)(1 - N) + \int_{K-S_0}^{\infty} x g(x) \mathrm d x\\
    & = (S_0 - K)(N\left(\frac{S_0 - K}{\sigma \sqrt{T}}\right)) + \int_{K-S_0}^{\infty} \frac{x}{\sqrt{2\pi} \sigma \sqrt{T}} e^{- \frac{x^2}{2\sigma^2 T}} \mathrm d x\\
    & = (S_0 - K) \, N\left(\frac{S_0 - K}{\sigma\sqrt T}\right) + \frac{\sigma\sqrt T}{\sqrt{2\pi}} \, e^{-\frac{(S_0 - K)^2}{2\sigma^2 T}}

\end{aligned}
$$

### <font color=blue> Solution 2(b). </font>

Calculate the partial derivatives of c(T, K):

$$
    c_T = \frac{\partial c}{\partial T}=\frac{\sigma}{2\sqrt{2\pi T}}e^{-\frac{\left(S_0-K\right)^2}{2\sigma^2T}},\ \ c_K = \frac{\partial c}{\partial K}=-N\left(\frac{S_0-K}{\sigma\sqrt{T}}\right),\ \ c_{KK} = \frac{\partial^2 c}{\partial K^2}=\frac{1}{\sigma\sqrt{2\pi T}}e^{-\frac{\left(S_0-K\right)^2}{2\sigma^2 T}}
$$

So 
$$
\begin{aligned}
    \frac {2c_T}{K^2c_{KK}} = \frac{\frac{\sigma}{2\sqrt{2\pi T}}}{K^2\frac{1}{\sigma\sqrt{2\pi T}}} = \frac{\sigma^2}{K^2}
\end{aligned}
$$